# 🎵 Spotify Wrapped Personalizado

In [23]:
import spotipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from spotipy.oauth2 import SpotifyOAuth
from main_functions import *

#Librerías de IA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

# Ajustar la fuente para soportar emojis
plt.rcParams['font.family'] = 'DejaVu Sans'

print("¡Entorno listo para Spotify Wrapped!")

¡Entorno listo para Spotify Wrapped!


🔐 Autenticación con Spotify

In [ ]:
# Llamar a la función de autenticación
sp = authenticate_spotify()

📊 Obtener los artistas y canciones más escuchadas

In [ ]:
# Obtener los 50 artistas más escuchados
top_artists = get_top_artists(sp, 50)

# Mostrar los X artistas más escuchados
artists_n = 20
print(f"🎤 Tus {artists_n} artistas más escuchados:")
display_top_artists(top_artists, artists_n)

In [ ]:
# Obtener las 50 canciones más escuchadas
top_tracks = get_top_tracks(sp, limit=50)

# Mostrar las X canciones más escuchadas
tracks_n = 20
print(f"\n🎵 Tus {tracks_n} canciones más escuchadas:")
display_top_tracks(top_tracks, display_limit=tracks_n)

📈 Análisis de la popularidad de las canciones

Este gráfico nos muestra que tan mainstream eres. O sin embargo tienes un gusto que no se adapta a lo popular.

In [ ]:
# Obtener la popularidad de las canciones
track_popularity = [track['popularity'] for track in top_tracks['items']]

# Visualizar la distribución de la popularidad
plt.figure(figsize=(10, 5))
sns.histplot(track_popularity, bins=20, kde=True, color='purple')
plt.title('🎵 Distribución de la Popularidad de tus Canciones Más Escuchadas')
plt.xlabel('Popularidad (0-100)')
plt.ylabel('Frecuencia')
plt.show()

📆 Años de lanzamiento de tus canciones más escuchadas

In [ ]:
# Obtener el año de lanzamiento de las canciones
track_years = [int(track['album']['release_date'][:4]) for track in top_tracks['items']]

# Crear un DataFrame para analizar la distribución de años
year_df = pd.DataFrame(track_years, columns=['year'])

# Crear una nueva columna para agrupar en décadas
year_df['decade'] = (year_df['year'] // 10) * 10  # Agrupar por décadas (ej. 1990, 2000, etc.)

# Graficar la distribución de las décadas
plt.figure(figsize=(10, 5))
sns.countplot(x='decade', data=year_df, palette='coolwarm', order=year_df['decade'].value_counts().index)
plt.title('📆 Años de Lanzamiento de tus Canciones Más Escuchadas (Por Décadas)')
plt.xlabel('Década')
plt.ylabel('Cantidad de Canciones')
plt.show()

Creamos un dataframe con los artistas más escuchados y su género musical

In [ ]:
# Crear un DataFrame con los artistas más escuchados
artist_data = pd.DataFrame(top_artists['items'])

# Eliminar columnas innecesarias y crear una copia explícita para evitar el warning
artist_data_clean = artist_data[['name', 'genres', 'followers', 'popularity']].copy()

# Reemplazar valores nulos en 'genres' por 'Desconocido' de forma segura
artist_data_clean.loc[:, 'genres'] = artist_data_clean['genres'].apply(lambda x: ', '.join(x) if x else 'Desconocido')

print(artist_data_clean.head())


In [ ]:
# Contar la frecuencia de cada artista
artist_counts = pd.Series([track['artists'][0]['name'] for track in top_tracks['items']]).value_counts()

# Mostrar un gráfico de pastel con los 10 artistas más repetidos
plt.figure(figsize=(8, 8))
artist_counts.head(10).plot.pie(autopct='%1.1f%%', startangle=90, cmap='Set2')
plt.title('🎤 Diversidad de Artistas (Top 10 más repetidos)')
plt.ylabel('')
plt.show()


Vamos a analizar los géneros más escuchados

In [ ]:
# 📊 Procesar las canciones y sus géneros
track_data = []

for idx, track in enumerate(top_tracks['items']):
    track_name = track['name']
    main_artist = track['artists'][0]['name']
    artist_id = track['artists'][0]['id']
    
    # Obtener detalles del artista para conocer los géneros
    artist_info = sp.artist(artist_id)
    genres = artist_info['genres']

    # Guardar la información (peso inverso: canción 1 tiene más peso)
    weight = 1 / (idx + 1)  
    for genre in genres:
        track_data.append({'track': track_name, 'artist': main_artist, 'genre': genre, 'weight': weight})

# Convertir a DataFrame
track_df = pd.DataFrame(track_data)

# 🧮 Calcular el peso total por género
genre_weights = track_df.groupby('genre')['weight'].sum().sort_values(ascending=False)

# 📊 Mostrar el Top 10 Géneros Ponderados
print(genre_weights.head(10))

# 📊 Visualizar los géneros más escuchados con peso
plt.figure(figsize=(12, 6))
sns.barplot(x=genre_weights.head(10).values, y=genre_weights.head(10).index, palette='viridis')
plt.title('Top 10 Géneros Más Escuchados (Ponderados por Frecuencia)')
plt.xlabel('Peso Total')
plt.ylabel('Género')
plt.show()

Ordenamos los artistas según sus seguidores e identificamos el más popular de nuestro top

In [ ]:
# Obtener los 10 artistas más escuchados
top_artists = get_top_artists(sp, limit=10)

# Crear DataFrame con los artistas más escuchados
artist_data = pd.DataFrame(top_artists['items'])

# Limpiar y organizar los datos de los artistas
artist_data_clean = artist_data[['name', 'genres', 'followers', 'popularity']].copy()

# Extraer el número total de seguidores
artist_data_clean['followers'] = artist_data_clean['followers'].apply(lambda x: x['total'])

# Mostrar los 10 artistas más escuchados
print("🎶 Tus 10 artistas más escuchados:")
print(artist_data_clean)

In [ ]:
# Artista con más seguidores
most_followed_artist = artist_data_clean.loc[artist_data_clean['followers'].idxmax()]
print(f"\n🎤 Artista con más seguidores: {most_followed_artist['name']} - {most_followed_artist['followers']} seguidores")

In [ ]:
# Gráfico de barras con los artistas más populares (de tus 10 más escuchados)
plt.figure(figsize=(12, 6))
sns.barplot(x=artist_data_clean['popularity'], y=artist_data_clean['name'], palette='magma')
plt.title('🔥 Popularidad de tus 10 Artistas Más Escuchados')
plt.xlabel('Popularidad (0-100)')
plt.ylabel('Artista')
plt.show()

IA QUE PREDICE SI TE GUSTARÁ UNA CANCIÓN O NO

In [25]:
# Obtener los 50 artistas más escuchados
top_artists = get_top_artists(sp, 50)

# Obtener las 50 canciones más escuchadas
top_tracks = get_top_tracks(sp, limit=50)

# Preparamos las características para cada canción
track_data = []
for track in top_tracks['items']:
    # Primero, obtenemos la información del artista de la canción
    artist = track['artists'][0]
    artist_name = artist['name']
    
    # Obtenemos los géneros del artista realizando una consulta adicional
    artist_info = sp.artist(artist['id'])  # Esto obtiene información detallada del artista
    genres = artist_info.get('genres', [])  # Extraemos los géneros, si existen

    # Ahora podemos preparar la información de la canción
    track_info = {
        'name': track['name'],
        'artist': artist_name,
        'popularity': track['popularity'],
        'release_year': int(track['album']['release_date'][:4]),
        'genres': genres if genres else ['No Genre'],  # Asignamos un valor predeterminado si no hay géneros
        # Otros datos que desees extraer
    }
    track_data.append(track_info)

# Convertir a DataFrame de Pandas
track_df = pd.DataFrame(track_data)

# Mostrar el DataFrame
print(track_df)

                                                 name                artist  \
0                                19 Dias y 500 Noches        Joaquín Sabina   
1                                       Qué Facilidad             Sen Senra   
2                                   BAILE INoLVIDABLE             Bad Bunny   
3                                             Goteras           Omar Montes   
4                                 Duro De Verdad pt.2          Los Sufridos   
5                                 Muriendo De Envidia            C. Tangana   
6                                           Hong Kong            C. Tangana   
7                                 Ya No Te Hago Falta             Sen Senra   
8                                               De Ti             Sen Senra   
9                                        Como Camarón                Estopa   
10                                              Tank!             SEATBELTS   
11                                Scotty Doesn't Kno

In [26]:
# Supongamos que tenemos un mecanismo para definir si te gustó una canción o no
track_df['liked'] = track_df['popularity'].apply(lambda x: 1 if x > 70 else 0)  # Ejemplo

# Convertir los géneros en variables dummy (OneHotEncoding)
# Primero verificamos si la columna 'genres' existe
if 'genres' in track_df.columns:
    track_df = track_df.join(track_df['genres'].str.join('|').str.get_dummies())
    
    # Eliminar la columna de géneros originales
    track_df.drop('genres', axis=1, inplace=True)

# Convertir la columna de 'artist' a valores numéricos (Label Encoding)
label_encoder = LabelEncoder()
track_df['artist'] = label_encoder.fit_transform(track_df['artist'])

# Eliminar la columna 'name' ya que no es útil para el modelo
track_df.drop('name', axis=1, inplace=True)

In [27]:
# Dividir los datos en entrenamiento y prueba
X = track_df.drop('liked', axis=1)  # Características (sin la columna 'liked')
y = track_df['liked']  # La etiqueta 'liked' es lo que queremos predecir

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo (Random Forest)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Hacer predicciones
y_pred = model.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy * 100:.2f}%")

# Predecir la probabilidad de que te guste una canción
probabilidad = model.predict_proba(X_test)

# Mostrar el porcentaje de probabilidad de que la canción te guste
# La columna 1 representa la probabilidad de que la canción te guste (etiqueta '1')
probabilidad_te_guste = probabilidad[:, 1]
print(f"Probabilidad de que te guste la canción: {probabilidad_te_guste[0] * 100:.2f}%")

Precisión del modelo: 90.00%
Probabilidad de que te guste la canción: 3.00%
